# Training and Serving CARET models using AI Platform Custom Containers and Cloud Run
## Overview

This notebook illustrates how to use [CARET](https://topepo.github.io/caret/) R package to build an ML model to estimate the baby's weight given a number of factors, using the [BigQuery natality dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&_ga=2.99329886.-1705629017.1551465326&_gac=1.109796023.1561476396.CI2rz-z4hOMCFc6RhQods4oEXA). We use [AI Platform Training](https://cloud.google.com/ml-engine/docs/tensorflow/training-overview) with **Custom Containers** to train the TensorFlow model at scale. Rhen use the [Cloud Run](https://cloud.google.com/run/docs/) to serve the trained model as a Web API for online predictions.

R is one of the most widely used programming languages for statistical modeling, which has a large and active community of data scientists and ML professional. 
With over 10,000 packages in the open-source repository of CRAN, R caters to all statistical data analysis applications, ML, and visualisation.


## Dataset
The dataset used in this tutorial is natality data, which describes all United States births registered in the 50 States, the District of Columbia, and New York City from 1969 to 2008, with more than 137 million records.
The dataset is available in [BigQuery public dataset](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&t=natality&page=table&_ga=2.99329886.-1705629017.1551465326&_gac=1.109796023.1561476396.CI2rz-z4hOMCFc6RhQods4oEXA). We use the data extracted from BigQuery and stored as CSV in Cloud Storage (GCS) in the [Exploratory Data Analysis](01_EDA-with-R-and-BigQuery) notebook.

In this notebook, we focus on Exploratory Data Analysis, while the goal is to predict the baby's weight given a number of factors about the pregnancy and the baby's mother.

## Objective
The goal of this tutorial is to:
1. Create a CARET regression model
2. Train the CARET model using on AI Platform Training with custom R container
3. Implement a Web API wrapper to the trained model using Plumber R package
4. Build Docker container image for the prediction Web API
5. Deploy the prediction Web API container image model on Cloud Run
6. Invoke the deployed Web API for predictions.
7. Use the AI Platform Notebooks to drive the workflow.



## Costs
This tutorial uses billable components of Google Cloud Platform (GCP):
1. Create a TensorFlow premade Estimator trainer using R interface
2. Train and export the Estimator on AI Platform Training using the cloudml APIs
3. Deploy the exported model to AI Platform prediction using the cloudml APIs
4. Invoke the deployed model API for predictions.
5. Use the AI Platform Notebooks to drive the workflow.


Learn about GCP pricing, use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## 0. Setup

In [ ]:
install.packages(c("caret"))

In [1]:
library(caret) # used to build a regression model

Loading required package: lattice
Loading required package: ggplot2


In [2]:
# Set the project id
project_id <- "r-on-gcp"

# Set yout GCS bucket
bucket <- "r_on_gcp" 

# Set your training and model deployment region
region <- 'europe-west1'

## 1. Building a CARET Regression Model

### 1.1. Load data

If you run the [Exploratory Data Analysis](01_EDA-with-R-and-BigQuery) Notebook, you should have the **train_data.csv** and **eval_data.csv** files uploaded to GCS. You can download them to train your model locally using the following cell. However, if you have the files available locally, you can skip the following cell.

In [3]:
dir.create(file.path('data'), showWarnings = FALSE)
gcs_data_dir <- paste0("gs://", bucket, "/data/*_data.csv")
command <- paste("gsutil cp -r", gcs_data_dir, "data/")
print(command)
system(command, intern = TRUE)

[1] "gsutil cp -r gs://r_on_gcp/data/*_data.csv data/"


character(0)

In [4]:
train_file <- "data/train_data.csv"
eval_file <- "data/eval_data.csv"
header <- c(
    "weight_pounds", 
    "is_male", "mother_age", "mother_race", "plurality", "gestation_weeks", 
    "mother_married", "cigarette_use", "alcohol_use", 
    "key")

target <- "weight_pounds"
key <- "key"
features <- setdiff(header, c(target, key))

train_data <- read.table(train_file, col.names = header, sep=",")
eval_data <- read.table(eval_file, col.names = header, sep=",")

### 1.2. Train the model
In this example, we will train an XGboost Tree model for regression.

In [5]:
trainControl <- trainControl(method = 'boot', number = 10)
hyper_parameters <- expand.grid(
    nrounds = 100,
    max_depth = 6,
    eta = 0.3,
    gamma = 0,
    colsample_bytree = 1,
    min_child_weight = 1,
    subsample = 1
)
  
print('Training the model...')

model <- train(
    y=train_data$weight_pounds, 
    x=train_data[, features], 
    preProc = c("center", "scale"),
    method='xgbTree', 
    trControl=trainControl,
    tuneGrid=hyper_parameters
)

print('Model is trained.')

[1] "Training the model..."
[1] "Model is trained."


### 1.2. Evaluate the model

In [6]:
eval(model)

eXtreme Gradient Boosting 

7742 samples
   8 predictor

Pre-processing: centered (4), scaled (4), ignore (4) 
Resampling: Bootstrapped (10 reps) 
Summary of sample sizes: 7742, 7742, 7742, 7742, 7742, 7742, ... 
Resampling results:

  RMSE      Rsquared   MAE      
  1.112589  0.3240922  0.8577826

Tuning parameter 'nrounds' was held constant at a value of 100
Tuning
 held constant at a value of 1
Tuning parameter 'subsample' was held
 constant at a value of 1

### 1.3. Save the trained model

In [7]:
model_dir <- "models"
model_name <- "caret_babyweight_estimator"

In [8]:
# Saving the trained model
dir.create(model_dir, showWarnings = FALSE)
dir.create(file.path(model_dir, model_name), showWarnings = FALSE)
saveRDS(model, file.path(model_dir, model_name, "trained_model.rds"))

### 1.4. Implementing a model prediction function
This is an implementation of wrapper function to the model to perform prediction. The function expects a list of instances in a JSON format, and returns a list of predictions (estimated weights). This prediction function implementation will be used when serving the model as a Web API for online predictions. 

In [9]:
xgbtree <- readRDS(file.path(model_dir, model_name, "trained_model.rds"))

estimate_babyweights <- function(instances_json){
    library("rjson")
    instances <- jsonlite::fromJSON(instances_json)
    df_instances <- data.frame(instances)
    # fix data types
    boolean_columns <- c("is_male", "mother_married", "cigarette_use", "alcohol_use")
    for(col in boolean_columns){
        df_instances[[col]] <- as.logical(df_instances[[col]])
    }
    
    estimates <- predict(xgbtree, df_instances)
    return(estimates) 
}

instances_json <- '
[
    {
        "is_male": "TRUE",
        "mother_age": 28,
        "mother_race": 8,
        "plurality": 1,
        "gestation_weeks":  28,
        "mother_married": "TRUE",
        "cigarette_use": "FALSE",
        "alcohol_use": "FALSE"
     },
    {
        "is_male": "FALSE",
        "mother_age": 38,
        "mother_race": 18,
        "plurality": 1,
        "gestation_weeks":  28,
        "mother_married": "TRUE",
        "cigarette_use": "TRUE",
        "alcohol_use": "TRUE"
     }
]
'

estimate <- round(estimate_babyweights(instances_json), digits = 2)
print(paste("Estimated weight(s):", estimate))

[1] "Estimated weight(s): 4.18" "Estimated weight(s): 3.7" 


## 3. Submit a Training Job to AI Platform with Custom Containers
In order to train your CARET model in at scale using AI Platform Training, you need to implement your training logic in an R script file, containerize it in a Docker image, and submit the Docker image to AI Platform Training.

The [src/caret/training](src/caret/training) directory includes the following code files:
1. [model_trainer.R](src/caret/training/model_trainer.R) - This is the implementation of the CARET model training logic.
1. [Dockerfile](src/caret/training/Dockerfile) - This is the definition of the Docker container image to run the **model_trainer.R** script.

To submit the training job with the custom container to AI Platform, you need to do the following steps:
2. **Build** a Docker container image with that runs the model_trainer.R
3. **Push** the Docker container image to **Container Registry**.
4. **Submit** an **AI Platform Training** job with the **custom container**.

### 3.1. Build and Push the Docker container image.
#### A - Build base image
This can take several minutes ...

In [10]:
# Create base image
base_image_url <- paste0("gcr.io/", project_id, "/caret_base")
print(base_image_url)

setwd("src/caret")
getwd()

print("Building the base Docker container image...")
command <- paste0("docker build -f Dockerfile --tag ", base_image_url, " ./")
print(command)
system(command, intern = TRUE)

print("Pushing the baseDocker container image...")
command <- paste0("gcloud docker -- push ", base_image_url)
print(command)
system(command, intern = TRUE)

setwd("../..")
getwd()

[1] "gcr.io/r-on-gcp/caret_base"


[1] "/home/jupyter/r-on-gcp/notebooks/src/caret"

[1] "Building the base Docker container image..."
[1] "docker build -f Dockerfile --tag gcr.io/r-on-gcp/caret_base ./"


[1] "Sending build context to Docker daemon  22.02kB\r\r"                                                                           
[2] "Step 1/2 : FROM gcr.io/deeplearning-platform-release/r-cpu"                                                                    
[3] " ---> ecba7177e2c2"                                                                                                            
[4] "Step 2/2 : RUN R -e \"install.packages(c('readr', 'caret', 'xgboost', 'rjson', 'plumber'), repos='http://cran.rstudio.com/')\""
[5] " ---> Using cache"                                                                                                             
[6] " ---> f233d2df59af"                                                                                                            
[7] "Successfully built f233d2df59af"                                                                                               
[8] "Successfully tagged gcr.io/r-on-gcp/caret_base:latest"

[1] "Pushing the baseDocker container image..."
[1] "gcloud docker -- push gcr.io/r-on-gcp/caret_base"


[1] "The push refers to repository [gcr.io/r-on-gcp/caret_base]"                                        
 [2] "f646cc3e71ef: Preparing"                                                                           
 [3] "0ce62bef372e: Preparing"                                                                           
 [4] "ac80f37c61e8: Preparing"                                                                           
 [5] "d35b581063f9: Preparing"                                                                           
 [6] "2cdc3d03b403: Preparing"                                                                           
 [7] "842c821f54eb: Preparing"                                                                           
 [8] "81c79e512da5: Preparing"                                                                           
 [9] "071617594623: Preparing"                                                                           
[10] "2eee26189b5e: Preparing"                                                                           
[11] "473763e23878: Preparing"                                                                           
[12] "8ff58362bc10: Preparing"                                                                           
[13] "c5890df75ecc: Preparing"                                                                           
[14] "1fb17ae4fc11: Preparing"                                                                           
[15] "0aaa26768f46: Preparing"                                                                           
[16] "fd5276389b8a: Preparing"                                                                           
[17] "56ac87b2a469: Preparing"                                                                           
[18] "140ce133886c: Preparing"                                                                           
[19] "b20d335c2af7: Preparing"                                                                           
[20] "e4dc9a88747b: Preparing"                                                                           
[21] "027d71a0bd0a: Preparing"                                                                           
[22] "2bcd744f68d7: Preparing"                                                                           
[23] "75e70aa52609: Preparing"                                                                           
[24] "dda151859818: Preparing"                                                                           
[25] "fbd2732ad777: Preparing"                                                                           
[26] "ba9de9d8475e: Preparing"                                                                           
[27] "842c821f54eb: Waiting"                                                                             
[28] "81c79e512da5: Waiting"                                                                             
[29] "071617594623: Waiting"                                                                             
[30] "2eee26189b5e: Waiting"                                                                             
[31] "473763e23878: Waiting"                                                                             
[32] "8ff58362bc10: Waiting"                                                                             
[33] "c5890df75ecc: Waiting"                                                                             
[34] "1fb17ae4fc11: Waiting"                                                                             
[35] "0aaa26768f46: Waiting"                                                                             
[36] "fd5276389b8a: Waiting"                                                                             
[37] "75e70aa52609: Waiting"                                                                             
[38] "56ac87b2a469: Waiting"                                                   

[1] "/home/jupyter/r-on-gcp/notebooks"

#### B - Build trainer image

In [11]:
training_image_url <- paste0("gcr.io/", project_id, "/", model_name, "_training")
print(training_image_url)

setwd("src/caret/training")
getwd()

print("Building the Docker container image...")
command <- paste0("docker build -f Dockerfile --tag ", training_image_url, " ./")
print(command)
system(command, intern = TRUE)

print("Pushing the Docker container image...")
command <- paste0("gcloud docker -- push ", training_image_url)
print(command)
system(command, intern = TRUE)

setwd("../../..")
getwd()

[1] "gcr.io/r-on-gcp/caret_babyweight_estimator_training"


[1] "/home/jupyter/r-on-gcp/notebooks/src/caret/training"

[1] "Building the Docker container image..."
[1] "docker build -f Dockerfile --tag gcr.io/r-on-gcp/caret_babyweight_estimator_training ./"


[1] "Sending build context to Docker daemon  10.75kB\r\r"                           
 [2] "Step 1/5 : FROM gcr.io/r-on-gcp/caret_base"                                    
 [3] " ---> f233d2df59af"                                                            
 [4] "Step 2/5 : RUN mkdir -p /root"                                                 
 [5] " ---> Using cache"                                                             
 [6] " ---> 0cb168ad1752"                                                            
 [7] "Step 3/5 : COPY model_trainer.R /root"                                         
 [8] " ---> Using cache"                                                             
 [9] " ---> e7b21ba3e110"                                                            
[10] "Step 4/5 : WORKDIR /root"                                                      
[11] " ---> Using cache"                                                             
[12] " ---> 1bf8e80133c9"                                                            
[13] "Step 5/5 : CMD [\"Rscript\", \"model_trainer.R\"]"                             
[14] " ---> Using cache"                                                             
[15] " ---> fc128c997331"                                                            
[16] "Successfully built fc128c997331"                                               
[17] "Successfully tagged gcr.io/r-on-gcp/caret_babyweight_estimator_training:latest"

[1] "Pushing the Docker container image..."
[1] "gcloud docker -- push gcr.io/r-on-gcp/caret_babyweight_estimator_training"


[1] "The push refers to repository [gcr.io/r-on-gcp/caret_babyweight_estimator_training]"               
 [2] "daf66802c11b: Preparing"                                                                           
 [3] "f646cc3e71ef: Preparing"                                                                           
 [4] "0ce62bef372e: Preparing"                                                                           
 [5] "ac80f37c61e8: Preparing"                                                                           
 [6] "d35b581063f9: Preparing"                                                                           
 [7] "2cdc3d03b403: Preparing"                                                                           
 [8] "842c821f54eb: Preparing"                                                                           
 [9] "81c79e512da5: Preparing"                                                                           
[10] "071617594623: Preparing"                                                                           
[11] "2eee26189b5e: Preparing"                                                                           
[12] "473763e23878: Preparing"                                                                           
[13] "8ff58362bc10: Preparing"                                                                           
[14] "c5890df75ecc: Preparing"                                                                           
[15] "1fb17ae4fc11: Preparing"                                                                           
[16] "0aaa26768f46: Preparing"                                                                           
[17] "fd5276389b8a: Preparing"                                                                           
[18] "56ac87b2a469: Preparing"                                                                           
[19] "140ce133886c: Preparing"                                                                           
[20] "b20d335c2af7: Preparing"                                                                           
[21] "e4dc9a88747b: Preparing"                                                                           
[22] "027d71a0bd0a: Preparing"                                                                           
[23] "2bcd744f68d7: Preparing"                                                                           
[24] "75e70aa52609: Preparing"                                                                           
[25] "dda151859818: Preparing"                                                                           
[26] "fbd2732ad777: Preparing"                                                                           
[27] "ba9de9d8475e: Preparing"                                                                           
[28] "071617594623: Waiting"                                                                             
[29] "2eee26189b5e: Waiting"                                                                             
[30] "473763e23878: Waiting"                                                                             
[31] "8ff58362bc10: Waiting"                                                                             
[32] "c5890df75ecc: Waiting"                                                                             
[33] "1fb17ae4fc11: Waiting"                                                                             
[34] "0aaa26768f46: Waiting"                                                                             
[35] "fd5276389b8a: Waiting"                                                                             
[36] "56ac87b2a469: Waiting"                                                                             
[37] "140ce133886c: Waiting"                                                                             
[38] "b20d335c2af7: Waiting"                                                   

[1] "/home/jupyter/r-on-gcp/notebooks"

#### C- Verifying uploaded images to Container Registry

In [12]:
command <- paste0("gcloud container images list --repository=gcr.io/", project_id)
system(command, intern = TRUE)

[1] "NAME"                                               
[2] "gcr.io/r-on-gcp/caret_babyweight_estimator_serving" 
[3] "gcr.io/r-on-gcp/caret_babyweight_estimator_training"
[4] "gcr.io/r-on-gcp/caret_base"

### 3.2. Submit an AI Plaform Training job with the custom container. 

In [13]:
job_name <- paste0("train_caret_contrainer_", format(Sys.time(), "%Y%m%d_%H%M%S"))

command = paste0("gcloud beta ml-engine jobs submit training ", job_name, 
  " --master-image-uri=", training_image_url,
  " --scale-tier=BASIC", 
  " --region=", region
)
print(command)

system(command, intern = TRUE)

[1] "gcloud beta ml-engine jobs submit training train_caret_contrainer_20190718_225807 --master-image-uri=gcr.io/r-on-gcp/caret_babyweight_estimator_training --scale-tier=BASIC --region=europe-west1"


[1] "jobId: train_caret_contrainer_20190718_225807"
[2] "state: QUEUED"

In [14]:
# Verify the trained model in GCS after the job finishes
model_name <- 'caret_babyweight_estimator'
gcs_model_dir <- paste0("gs://", bucket, "/models/", model_name)
command <- paste0("gsutil ls ", gcs_model_dir)
system(command, intern = TRUE)

[1] "gs://r_on_gcp/models/caret_babyweight_estimator/trained_model.rds"

## 4. Deploy the trained model to Cloud Run
In order to serve the trained CARET model as a Web API, you need to wrap it with a prediction function, as serve this prediction function as a REST API. Then you containerize this Web API and deploy it in Cloud Run.

The [src/caret/serving](src/caret/serving) directory includes the following code files:
1. [model_prediction.R](src/caret/serving/model_prediction.R) - This script downloads the trained model from GCS and loads (only once). It includes **estimate** function, which accepts instances in JSON format, and return the of baby weight estimate for each instance.
2. [model_api.R](src/caret/serving/model_prediction.R) - This is a [plumber](https://www.rplumber.io/) Web API that runs  **model_prediction.R**.
3. [Dockerfile](src/caret/serving/Dockerfile) - This is the definition of Docker container image that runs the **model_api.R**

To deploy the prediction Web API to Cloud Run, you need to do the following steps:
1. **Build** the Docker container image for the prediction API.
2. **Push** the Docker container image to **Cloud Registry**.
3. **Deploy** the Docker container to **Cloud Run**. 



### (Optional) 4.0. Upload the trained model to GCS
If you train your model using the model_trainer.R in AI Platform, it will upload the saved model to GCS. However, if you only train your model locally and have your saved model locally, you need to upload it to GCS.

In [15]:
model_name <- 'caret_babyweight_estimator'
gcs_model_dir = paste0("gs://", bucket, "/models/", model_name, "/")
command <- paste0("gsutil cp -r models/", model_name ,"/* ",gcs_model_dir)
print(command)
system(command, intern = TRUE)

[1] "gsutil cp -r models/caret_babyweight_estimator/* gs://r_on_gcp/models/caret_babyweight_estimator/"


character(0)

### 4.1. Build and Push prediction Docker container image

In [16]:
serving_image_url <- paste0("gcr.io/", project_id, "/", model_name, "_serving")
print(serving_image_url)

setwd("src/caret/serving")
getwd()

print("Building the Docker container image...")
command <- paste0("docker build -f Dockerfile --tag ", serving_image_url, " ./")
print(command)
system(command, intern = TRUE)

print("Pushing the Docker container image...")
command <- paste0("gcloud docker -- push ", serving_image_url)
print(command)
system(command, intern = TRUE)

setwd("../../..")
getwd()

[1] "gcr.io/r-on-gcp/caret_babyweight_estimator_serving"


[1] "/home/jupyter/r-on-gcp/notebooks/src/caret/serving"

[1] "Building the Docker container image..."
[1] "docker build -f Dockerfile --tag gcr.io/r-on-gcp/caret_babyweight_estimator_serving ./"


[1] "Sending build context to Docker daemon  8.704kB\r\r"                          
 [2] "Step 1/8 : FROM gcr.io/r-on-gcp/caret_base"                                   
 [3] " ---> f233d2df59af"                                                           
 [4] "Step 2/8 : RUN mkdir -p /root"                                                
 [5] " ---> Using cache"                                                            
 [6] " ---> 0cb168ad1752"                                                           
 [7] "Step 3/8 : COPY model_prediction.R /root"                                     
 [8] " ---> Using cache"                                                            
 [9] " ---> bd0ce1137523"                                                           
[10] "Step 4/8 : COPY model_api.R /root"                                            
[11] " ---> Using cache"                                                            
[12] " ---> 536f0568cfa2"                                                           
[13] "Step 5/8 : WORKDIR /root"                                                     
[14] " ---> Using cache"                                                            
[15] " ---> 2ff7f1b3cb1b"                                                           
[16] "Step 6/8 : ENV PORT 8080"                                                     
[17] " ---> Using cache"                                                            
[18] " ---> 035dde41d109"                                                           
[19] "Step 7/8 : EXPOSE 8080"                                                       
[20] " ---> Using cache"                                                            
[21] " ---> 7909af045903"                                                           
[22] "Step 8/8 : ENTRYPOINT [\"Rscript\", \"model_api.R\"]"                         
[23] " ---> Using cache"                                                            
[24] " ---> 495b8035a034"                                                           
[25] "Successfully built 495b8035a034"                                              
[26] "Successfully tagged gcr.io/r-on-gcp/caret_babyweight_estimator_serving:latest"

[1] "Pushing the Docker container image..."
[1] "gcloud docker -- push gcr.io/r-on-gcp/caret_babyweight_estimator_serving"


[1] "The push refers to repository [gcr.io/r-on-gcp/caret_babyweight_estimator_serving]"                
 [2] "3d907e7720cb: Preparing"                                                                           
 [3] "a33afe959f67: Preparing"                                                                           
 [4] "f646cc3e71ef: Preparing"                                                                           
 [5] "0ce62bef372e: Preparing"                                                                           
 [6] "ac80f37c61e8: Preparing"                                                                           
 [7] "d35b581063f9: Preparing"                                                                           
 [8] "2cdc3d03b403: Preparing"                                                                           
 [9] "842c821f54eb: Preparing"                                                                           
[10] "81c79e512da5: Preparing"                                                                           
[11] "071617594623: Preparing"                                                                           
[12] "2eee26189b5e: Preparing"                                                                           
[13] "473763e23878: Preparing"                                                                           
[14] "8ff58362bc10: Preparing"                                                                           
[15] "c5890df75ecc: Preparing"                                                                           
[16] "1fb17ae4fc11: Preparing"                                                                           
[17] "0aaa26768f46: Preparing"                                                                           
[18] "fd5276389b8a: Preparing"                                                                           
[19] "56ac87b2a469: Preparing"                                                                           
[20] "140ce133886c: Preparing"                                                                           
[21] "b20d335c2af7: Preparing"                                                                           
[22] "e4dc9a88747b: Preparing"                                                                           
[23] "027d71a0bd0a: Preparing"                                                                           
[24] "2bcd744f68d7: Preparing"                                                                           
[25] "75e70aa52609: Preparing"                                                                           
[26] "dda151859818: Preparing"                                                                           
[27] "fbd2732ad777: Preparing"                                                                           
[28] "ba9de9d8475e: Preparing"                                                                           
[29] "c5890df75ecc: Waiting"                                                                             
[30] "1fb17ae4fc11: Waiting"                                                                             
[31] "2cdc3d03b403: Waiting"                                                                             
[32] "0aaa26768f46: Waiting"                                                                             
[33] "842c821f54eb: Waiting"                                                                             
[34] "fd5276389b8a: Waiting"                                                                             
[35] "56ac87b2a469: Waiting"                                                                             
[36] "81c79e512da5: Waiting"                                                                             
[37] "140ce133886c: Waiting"                                                                             
[38] "071617594623: Waiting"                                                   

[1] "/home/jupyter/r-on-gcp/notebooks"

In [17]:
command <- paste0("gcloud container images list --repository=gcr.io/", project_id)
system(command, intern = TRUE)

[1] "NAME"                                               
[2] "gcr.io/r-on-gcp/caret_babyweight_estimator_serving" 
[3] "gcr.io/r-on-gcp/caret_babyweight_estimator_training"
[4] "gcr.io/r-on-gcp/caret_base"

### 4.2. Deploy prediction container to Cloud Run

In [18]:
service_name <- "caret-babyweight-estimator"
command <- paste(
    "gcloud beta run deploy", service_name,
    "--image", serving_image_url,
    "--platform managed",
    "--allow-unauthenticated",
    "--region", region
)

print(command)
system(command, intern = TRUE)

[1] "gcloud beta run deploy caret-babyweight-estimator --image gcr.io/r-on-gcp/caret_babyweight_estimator_serving --platform managed --allow-unauthenticated --region europe-west1"


## 5. Invoke the Model API for Predictions

When the **caret-babyweight-estimator** service is deployed to Cloud Run:
1. Go to Cloud Run in the [Cloud Console](https://console.cloud.google.com/run/).
2. Select the **caret-babyweight-estimator** service.
3. Copy the service URL, and use it to update the **url** variable in the following cell.

In [19]:
# Update to the deployed service URL
url <- "https://caret-babyweight-estimator-qeht62dxla-ew.a.run.app/"
endpoint <- "estimate"

In [20]:
instances_json <- '
[
    {
        "is_male": "TRUE",
        "mother_age": 28,
        "mother_race": 8,
        "plurality": 1,
        "gestation_weeks":  28,
        "mother_married": "TRUE",
        "cigarette_use": "FALSE",
        "alcohol_use": "FALSE"
     },
    {
        "is_male": "FALSE",
        "mother_age": 38,
        "mother_race": 18,
        "plurality": 1,
        "gestation_weeks":  28,
        "mother_married": "TRUE",
        "cigarette_use": "TRUE",
        "alcohol_use": "TRUE"
     }
]
'

In [22]:
library("httr")
full_url <- paste0(url, endpoint)
response <- POST(full_url, body = instances_json)
estimates <- content(response)
print(paste("Estimated weight(s):", estimate))

[1] "Estimated weight(s): 4.18" "Estimated weight(s): 3.7" 


# License

Authors: Daniel Sparing & Khalid Salama

---
**Disclaimer**: This is not an official Google product. The sample code provided for an educational purpose.

---

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.